In [1]:
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1011. Emission Mixtures/P1011.3. Analyses/P1011.3.2. Biomarker Distribution Analysis/Output')
cur_date = "050223"

library(readxl)
library(tidyverse)
library(reshape2)
library(stats)
library(multcomp)

# reading in files
proteomics_5_df = data.frame(read_excel("Input/Proteomics_Data_5ug_040423.xlsx", sheet = 2))
proteomics_25_df = data.frame(read_excel("Input/Proteomics_Data_25ug_040423.xlsx", sheet = 2))
demographics_df = data.frame(read_excel("Input/Subject_Info_031723.xlsx", sheet = 2))

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


Loading required package: mvtnorm

Loading required package: survival

Loading required package: TH.data

Loading required package: MASS


Attaching package: ‘MASS’


The following object is masked from ‘package:dplyr’:

    select



Attaching package: ‘TH.data’


The following object is masked from ‘package:MASS’:

    geyser




Testing for statistical differences by comparing a protein expression between burn conditions (control, smoldering or flaming). (The condensate and concentration would remain consistent). Protein expression (intensity) was pseudo log transformed to reduce skew.

This time sex will be added to models as a covariate. I couldn't find a non-parametric test in R that would allow for the addition of a covariate, therefore crude ANOVA models/ Tukey's post hoc test will be run first and then ANCOVA models/ Tukey's post hoc test that include sex.

In [2]:
head(proteomics_25_df)
head(demographics_df)

,Subject_No,Subject_ID,Protein_Accession,Gene_Name,Description,Condensate,Burn_Condition,Concentration,Intensity,Intensity_pslog2
,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,1,F_1,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,1488600000,30.47131
2,2,M_2,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,3806600000,31.82586
3,3,M_3,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,3900000000,31.86083
4,4,F_4,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,5373200000,32.32313
5,5,F_5,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,2595200000,31.27320
6,6,M_6,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,5780700000,32.42860


,Original_Subject_ID,Subject_ID,Subject_No,Sex,Age,Race,Ethnicity
,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>
1,68J,F_1,1,F,18,B,NH
2,53K,M_2,2,M,32,B,NH
3,21O,M_3,3,M,34,W,H
4,63O,F_4,4,F,32,W,H
5,67L,F_5,5,F,17,W,H
6,57N,M_6,6,M,59,W,NH


In [3]:
# combining data, but first making sex a factor
demographics_df = demographics_df %>%
    mutate(Sex = relevel(factor(ifelse(Sex == "M", 1, 0)), ref = "0"))

proteomics_25_sex_df = inner_join(proteomics_25_df[,-c(4:5)], demographics_df[,c(2,4)]) %>%
    # scaling the concentration values by protein normalizes the distribution
    group_by(Protein_Accession) %>%
    mutate(Scaled_Intensity = scale(Intensity_pslog2))
proteomics_5_sex_df = inner_join(proteomics_5_df[,-c(4:5)], demographics_df[,c(2,4)]) %>%
    # scaling the concentration values by protein normalizes the distribution
    group_by(Protein_Accession) %>%
    mutate(Scaled_Intensity = scale(Intensity_pslog2))

# putting burn condition into a factor
proteomics_25_sex_df$Burn_Condition = factor(proteomics_25_sex_df$Burn_Condition, levels = c("PBS", "S", "F"))
proteomics_5_sex_df$Burn_Condition = factor(proteomics_5_sex_df$Burn_Condition, levels = c("PBS", "S", "F"))

head(proteomics_25_sex_df)

Joining, by = "Subject_ID"
Joining, by = "Subject_ID"


Subject_No,Subject_ID,Protein_Accession,Condensate,Burn_Condition,Concentration,Intensity,Intensity_pslog2,Sex,Scaled_Intensity
<dbl>,<chr>,<chr>,<chr>,<fct>,<dbl>,<dbl>,<dbl>,<fct>,"<dbl[,1]>"
1,F_1,CO3_HUMAN,C,F,25,1488600000,30.47131,0,-1.29273172
2,M_2,CO3_HUMAN,C,F,25,3806600000,31.82586,1,-0.04856119
3,M_3,CO3_HUMAN,C,F,25,3900000000,31.86083,1,-0.01643970
4,F_4,CO3_HUMAN,C,F,25,5373200000,32.32313,0,0.40819624
5,F_5,CO3_HUMAN,C,F,25,2595200000,31.27320,0,-0.55618474
6,M_6,CO3_HUMAN,C,F,25,5780700000,32.42860,1,0.50506520


In [4]:
# contrasts show what compartments are being compared in anova
# these comparisons aren't what we want so I changed them in the function below
contrasts(proteomics_25_sex_df$Burn_Condition)

,S,F
PBS,0,0
S,1,0
F,0,1


               Table of Contrasts 
>               PBS |  S  |  F  |  Sum
>  - Contrast 1 |  -1  |  1  |  0  |   0
>  - Contrast 2 |  -1  |  0  |  1  |   0

> - Contrast 1: compares PBS to smoldering
> - Contrast 2: compares PBS to flaming

In [5]:
get_anova = function(df, conc){
    # """
    # Running anova/ tukey's tests after filtering for a protein and condensate using a loop. 
    # Ultimately using this test to compare protein expression (control vs. flaming burn condition).

    # :param: protein dataframe, concentration
    # :output: a dataframe containing the protein, condensate, comparison, stat, p value, p adj

    # """
    
    # first filtering the df and iterating through the variables
    proteins = unique(df$Protein_Accession)
    condensates = c("C", "P")
    
    tukey_df = data.frame()
    for(i in 1:length(proteins)){
        for(j in 1:length(condensates)){
            filtered_df = df %>%
                filter(Protein_Accession == proteins[i], Condensate == c(condensates[j], "PBS"))
            
            # changing contrasts
            contrasts(filtered_df$Burn_Condition) = cbind(c(-1,1,0), c(-1,0,1)) # meaning is specified above
            # ANOVA
            anova = aov(Scaled_Intensity ~ Burn_Condition, data = filtered_df)

            # Tukey's post hoc test
            tukeys_anova = summary(glht(anova, linfct = mcp(Burn_Condition = "Tukey")), test = adjusted("none"))
            tukeys_test = tukeys_anova$test
            values_df = data.frame("ANOVA", condensates[j], proteins[i], conc, tukeys_test$tstat, 
                                   tukeys_test$pvalues) %>%
                rownames_to_column(var = "Comparison")

            # saving values
            tukey_df = rbind(tukey_df, values_df)
        }
    }
    
    # changing col names
    colnames(tukey_df)[2:7] = c("Model", "Condensate", "Protein_Accession", "Concentration", "Statistic", 
                                "P Value")
    
 
    # calculating padj values
    PAdj = c()
    for(j in 1:length(condensates)){
        filtered_df = tukey_df %>%
            filter(Condensate == condensates[j])
        padj = p.adjust(as.numeric(as.character(filtered_df$`P Value`)), method = "fdr")
        PAdj = c(PAdj, padj)

    }
    
    tukey_df$`P Adj` = PAdj
    
    return(tukey_df)
}

# calling fn
proteomics_25_anova_tukey = get_anova(proteomics_25_sex_df, 25)
proteomics_5_anova_tukey = get_anova(proteomics_5_sex_df, 5)

Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in cov2cor(covm):
“diag(.) had 0 or NA entries; non-finite result is doubtful”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfe

Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in cov2cor(covm):
“diag(.) had 0 or NA entries; non-finite result is doubtful”
Warning message in summary.lm(object, ...):
“essentially perfe

Warning message in cov2cor(covm):
“diag(.) had 0 or NA entries; non-finite result is doubtful”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in cov2cor(covm):
“diag(.) had 0 or NA entries; non-finite result is doubtful”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in cov2cor(covm):
“diag(.) had 0 or NA entries; non-finite result is doubtful”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in cov2cor(covm):
“diag(.) had 0 or NA entries; non-finite result is doubtful”
Warning message in summary.lm(object, ...):
“essentially

Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in cov2cor(covm):
“diag(.) had 0 or NA entries; non-finite result is doubtful”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in cov2cor(covm):
“diag(.) had 0 or NA entries; non-finite result is doubtful”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect

Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in cov2cor(covm):
“diag(.) had 0 or NA entries; non-finite result is doubtful”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in cov2cor(covm):
“diag(.) had 0 or NA entries; non-finite result is doubtful”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in cov2cor(covm):
“diag(.) had 0 or NA entries; non-finite result is doubtful”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially per

...

Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in cov2cor(covm):
“diag(.) had 0 or NA entries; non-finite result is doubtful”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in cov2cor(covm):
“diag(.) had 0 or NA entries; non-finite result is doubtful”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in cov2cor(covm):
“diag(.) had 0 or NA entries; non-finite result is doubtful”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially per

Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect f

Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in cov2cor(covm):
“diag(.) had 0 or NA entries; non-finite result is doubtful”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfe

Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in cov2cor(covm):
“diag(.) had 0 or NA entries; non-finite result is doubtful”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in cov2cor(covm):
“diag(.) had 0 or NA entries

Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in cov2cor(covm):
“diag(.) had 0 or NA entries; non-finite result is doubtful”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfe

In [6]:
get_ancova = function(df, conc){
    # """
    # Running anova/ tukey's tests after filtering for protein and condensate using a loop. 
    # Ultimately using this test to compare protein expression (control vs. smoldering vs. flaming burn condition).
    # Adding in sex as a covariate.

    # :param: subsetted dataframe, concetration
    # :output: a dataframe containing the protein, condensate, comparison, conc, stat, p value, p adj

    # """
    
    # first filtering the df and iterating through the variables
    proteins = unique(df$Protein_Accession)
    condensates = c("C", "P")
    
    tukey_df = data.frame()
    for(i in 1:length(proteins)){
        for(j in 1:length(condensates)){
            filtered_df = df %>%
                filter(Protein_Accession == proteins[i], Condensate == c(condensates[j], "PBS"))

            # changing contrasts
            contrasts(filtered_df$Burn_Condition) = cbind(c(-1,1,0), c(-1,0,1)) # meaning is specified above
            # ANCOVA
            ancova = aov(Scaled_Intensity ~ Sex + Burn_Condition, data = filtered_df)

            # Tukey's post hoc test
            tukeys_ancova = summary(glht(ancova, linfct = mcp(Burn_Condition = "Tukey")), test = adjusted("none"))
            tukeys_test = tukeys_ancova$test
            values_df = data.frame("ANCOVA", condensates[j], proteins[i], conc, tukeys_test$tstat, 
                                   tukeys_test$pvalues) %>%
                rownames_to_column(var = "Comparison")

            # saving values
            tukey_df = rbind(tukey_df, values_df)
        }
    }
    
    # changing col names
    colnames(tukey_df)[2:7] = c("Model", "Condensate", "Protein_Accession", "Concentration", "Statistic", 
                                "P Value")
    
 
    # calculating padj values
    PAdj = c()
    for(j in 1:length(condensates)){
        filtered_df = tukey_df %>%
            filter(Condensate == condensates[j])
        padj = p.adjust(as.numeric(as.character(filtered_df$`P Value`)), method = "fdr")
        PAdj = c(PAdj, padj)

    }
    
    tukey_df$`P Adj` = PAdj
    
    return(tukey_df)
}

# calling fn
proteomics_25_ancova_tukey = get_ancova(proteomics_25_sex_df, 25)
proteomics_5_ancova_tukey = get_ancova(proteomics_5_sex_df, 5)

Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in cov2cor(covm):
“diag(.) had 0 or NA entries; non-finite result is doubtful”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfe

Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in cov2cor(covm):
“diag(.) had 0 or NA entries; non-finite result is doubtful”
Warning message in summary.lm(object, ...):
“essentially perfe

Warning message in cov2cor(covm):
“diag(.) had 0 or NA entries; non-finite result is doubtful”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in cov2cor(covm):
“diag(.) had 0 or NA entries; non-finite result is doubtful”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in cov2cor(covm):
“diag(.) had 0 or NA entries; non-finite result is doubtful”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in cov2cor(covm):
“diag(.) had 0 or NA entries; non-finite result is doubtful”
Warning message in summary.lm(object, ...):
“essentially

Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in cov2cor(covm):
“diag(.) had 0 or NA entries; non-finite result is doubtful”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in cov2cor(covm):
“diag(.) had 0 or NA entries; non-finite result is doubtful”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect

Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in cov2cor(covm):
“diag(.) had 0 or NA entries; non-finite result is doubtful”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in cov2cor(covm):
“diag(.) had 0 or NA entries; non-finite result is doubtful”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in cov2cor(covm):
“diag(.) had 0 or NA entries; non-finite result is doubtful”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially per

...

Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in cov2cor(covm):
“diag(.) had 0 or NA entries; non-finite result is doubtful”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in cov2cor(covm):
“diag(.) had 0 or NA entries; non-finite result is doubtful”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in cov2cor(covm):
“diag(.) had 0 or NA entries; non-finite result is doubtful”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially per

Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect f

Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in cov2cor(covm):
“diag(.) had 0 or NA entries; non-finite result is doubtful”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfe

Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in cov2cor(covm):
“diag(.) had 0 or NA entries; non-finite result is doubtful”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in cov2cor(covm):
“diag(.) had 0 or NA entries

Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in cov2cor(covm):
“diag(.) had 0 or NA entries; non-finite result is doubtful”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(model):
“essentially perfe

In [7]:
anova_ancova_df = rbind(proteomics_25_anova_tukey, proteomics_5_anova_tukey, proteomics_25_ancova_tukey,
                      proteomics_5_ancova_tukey) %>%
    # changing condensates so they're more legible
    mutate(Condensate = ifelse(Condensate == "C", "Cardboard",
                               ifelse(Condensate == "P", "Plastic", "PBS")))
head(anova_ancova_df)

,Comparison,Model,Condensate,Protein_Accession,Concentration,Statistic,P Value,P Adj
,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,S - PBS,ANOVA,Cardboard,CO3_HUMAN,25,0.40120170,0.7021695,0.9886984
2,F - PBS,ANOVA,Cardboard,CO3_HUMAN,25,-0.08442488,0.9354651,1.0000000
3,F - S,ANOVA,Cardboard,CO3_HUMAN,25,-0.48562658,0.6444557,0.9239879
4,S - PBS,ANOVA,Plastic,CO3_HUMAN,25,0.27620345,0.7916591,0.2481598
5,F - PBS,ANOVA,Plastic,CO3_HUMAN,25,0.25703616,0.8057382,0.4700276
6,F - S,ANOVA,Plastic,CO3_HUMAN,25,-0.01916729,0.9853291,0.7078710


In [8]:
# exporting
write.csv(anova_ancova_df, paste0(Output,"/", cur_date, "_Proteomics_ANOVA_ANCOVA.csv"), row.names = FALSE)